In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler

In [ ]:
swat_data = pd.read_excel("../data/SWaT/SWaT_Dataset_Normal_v1.xlsx", skiprows=1)

In [ ]:
swat_data.head()

In [ ]:
swat_data = swat_data.set_index(" Timestamp")

In [ ]:
swat_data.to_parquet("../data/SWaT/swat.parquet.gzip", compression="gzip", engine="pyarrow")

In [ ]:
swat_data_parquet = pd.read_parquet("../data/SWaT/swat.parquet.gzip", engine="pyarrow")

In [ ]:
swat_data_parquet.head()

In [ ]:
swat_data_parquet.drop("Normal/Attack", axis=1, inplace=True)

# Throw out constant signals

In [ ]:
threshold = 0.0
swat_data_parquet_nonconstant = swat_data_parquet.drop(
    swat_data_parquet.std()[swat_data_parquet.std() <= threshold].index.values, axis=1
)

In [ ]:
swat_data_parquet_nonconstant.describe()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
swat_data_parquet_scaled = scaler.fit_transform(swat_data_parquet_nonconstant)

In [ ]:
swat_data_parquet_scaled_df = pd.DataFrame(
    swat_data_parquet_scaled, columns=swat_data_parquet_nonconstant.columns
)

In [ ]:
swat_data_parquet_scaled_df.head()

In [ ]:
swat_torch = torch.from_numpy(swat_data_parquet_scaled_df.values).float()

In [ ]:
swat_torch.shape

# Create snapshots of rolling windows
admittedly in an inefficient way

In [ ]:
# create list of tensors that cover a time window of 100 timesteps each
# slide the window by 10 timesteps
# the last window is dropped if it is smaller than 100 timesteps

windows = []

for i in range(0, swat_torch.shape[0] - 100, 10):
    windows.append(swat_torch[i : i + 100])

In [ ]:
swat_torch_snapshots = torch.stack(windows)
swat_torch_snapshots.shape

In [ ]:
swat_torch_snapshots = swat_torch_snapshots.permute(0, 2, 1)

In [ ]:
swat_torch_snapshots.shape

In [ ]:
torch.save(swat_torch_snapshots, "../data/SWaT/swat_snapshots.pt")

# Build ground truth adjacency matrix

In [ ]:
swat_data_parquet_scaled_df.columns

In [ ]:
p1 = torch.ones(4, 4)
p2 = torch.ones(7, 7)
p3 = torch.ones(9, 9)
p4 = torch.ones(6, 6)
p5 = torch.ones(12, 12)
p6 = torch.ones(2, 2)

In [ ]:
normal_adj = torch.block_diag(*[p1, p2, p3, p4, p5, p6])

In [ ]:
normal_adj.shape

In [ ]:
normal_adj

In [ ]:
torch.save(normal_adj, "../data/SWaT/normal_adj.pt")